In [1]:
from Model_Training import Model_Training
import plotly.express as px
import warnings
import pandas as pd
from EmpiricDistribution import EmpiricDistribution
import pickle # Para guardar los modelos entrenados
from datetime import time 
import matplotlib.pyplot as plt
import numpy as np
import random
from sympy import Symbol
from datetime import time

model = Model_Training()

In [ ]:
#Uncomment this cell if you dont have the .pkl files


#model.generate_models()

c:\Users\crdie\Documents\Tec\5Semestre\Optimizacion\DiscreteSimulation-TransporTec\Cleaner.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['trayecto'] = df_final['inicio'] + '-'+ df_final['final']


Generando histograma para A2-GLAXO_0700-0930
Modelo A2-GLAXO_0700-0930 entrenado. Número de segmentos (inversas): 17
Generando histograma para A2-GLAXO_1330-1800
Error: No valid symbolic solution found (likely m=0) for interval 25. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 28. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 29. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 30. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 31. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 32. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 33. Using x_max as inverse.
Modelo A2-GLAXO_1330-1800 entrenado. Número de segmentos (inversas): 35
Generando histograma para A2-GLAXO_0600-0700
Modelo A2-GLAXO_0600-0700 entrenado. Número de segmentos (inver

In [5]:
#Uncomment this cell if you dont have the simulation_results_completa.csv

#model.generate_simulation()

In [4]:
#model.show_results()

In [6]:
# Cargar los resultados de la simulación
df_simulacion = pd.read_csv('simulation_results_completa.csv')

# Análisis del Tiempo de Viaje (Minutos)
time_analysis = df_simulacion.groupby('trayecto')['minutos_viaje_simulado'].agg(
    Media_Tiempo=np.mean,
    P95_Tiempo=lambda x: np.quantile(x, 0.95), # Usamos una lambda para el cuantil de forma segura
    Desviacion_Estandar_Tiempo=np.std
).reset_index()

# Análisis de Pasajeros (Usuarios)
pax_analysis = df_simulacion.groupby('trayecto')['usuarios_simulado'].agg(
    Media_Usuarios=np.mean,
    Max_Usuarios_Simulados=np.max
).reset_index()

# Reporte
reporte_final = pd.merge(time_analysis, pax_analysis, on='trayecto')

reporte_final['Media_Tiempo'] = reporte_final['Media_Tiempo'].round(2)
reporte_final['P95_Tiempo'] = reporte_final['P95_Tiempo'].round(2)
reporte_final['Desviacion_Estandar_Tiempo'] = reporte_final['Desviacion_Estandar_Tiempo'].round(2)
reporte_final['Media_Usuarios'] = reporte_final['Media_Usuarios'].round(2)

reporte_final.columns = [
    'Segmento',
    'Tiempo Promedio (min)',
    'Tiempo P95 (min) - Planificación',
    'Desviación Estándar (min)',
    'Pasajeros Promedio (Usuarios)',
    'Pasajeros Máximo Simulado (Usuarios)'
]

# Exportar el reporte final
output_report_file = 'reporte_montecarlo_final.csv'
reporte_final.to_csv(output_report_file, index=False)

print(f"\nReporte de Monte Carlo generado en '{output_report_file}'")
print("\n--- Vista Previa del Reporte de Optimización ---")
print(reporte_final)


Reporte de Monte Carlo generado en 'reporte_montecarlo_final.csv'

--- Vista Previa del Reporte de Optimización ---
                  Segmento  Tiempo Promedio (min)  \
0       A2-GLAXO_0600-0700                   8.25   
1       A2-GLAXO_0700-0930                   8.68   
2       A2-GLAXO_0930-1330                   7.30   
3       A2-GLAXO_1330-1800                   7.77   
4       A2-GLAXO_1830-2215                   8.35   
5      DISNEY-A2_0600-0700                  14.46   
6      DISNEY-A2_0700-0930                  15.87   
7      DISNEY-A2_0930-1330                  14.28   
8      DISNEY-A2_1330-1800                  14.48   
9      DISNEY-A2_1830-2215                   8.39   
10  GLAXO-DISNEY_0600-0700                   8.94   
11  GLAXO-DISNEY_0700-0930                  10.66   
12  GLAXO-DISNEY_0930-1330                  11.83   
13  GLAXO-DISNEY_1330-1800                  11.54   
14  GLAXO-DISNEY_1830-2215                   8.30   

    Tiempo P95 (min) - Planificaci

C:\Users\crdie\AppData\Local\Temp\ipykernel_30004\2906294748.py:5: FutureWarning: The provided callable <function mean at 0x00000234C9107100> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  time_analysis = df_simulacion.groupby('trayecto')['minutos_viaje_simulado'].agg(
C:\Users\crdie\AppData\Local\Temp\ipykernel_30004\2906294748.py:5: FutureWarning: The provided callable <function std at 0x00000234C9107240> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  time_analysis = df_simulacion.groupby('trayecto')['minutos_viaje_simulado'].agg(
C:\Users\crdie\AppData\Local\Temp\ipykernel_30004\2906294748.py:12: FutureWarning: The provided callable <function mean at 0x00000234C9107100> is currently using SeriesGroupBy.mean. In a future version of pand